## Bonusový úkol č. 2 - stahování dat z webového zdroje
Vytvořte funkci **sync()**, která získá kompletní seznam produktů (tj. včetně dalších stránek) dostupných v kategorii
https://www.alza.cz/bezzrcadlovky-bez-objektivu/18863907.htm
a u každého produktu zjistí jeho aktuální cenu a stav skladu.
Funkce bude uchovávat získané informace a historii změn v relační databázi SQLLite3 obsahující dvě tabulky:  
* tabulku `products` a  
* tabulku `products_history`.

Struktura obou tabulek je shodná a obsahuje následující sloupce:  
* `id` TEXT - id produktu, např. OS072i1l1 (viz data-impression-id),  
* `url` TEXT - url produktu k kterému se vztahuje cena (pouze část path, viz ukázka na konci),  
* `title` TEXT - název produktu,  
* `price` DECIMAL - cena produktu s DPH k danému datu,   
* `stock_state` TEXT - stav skladu k danému datu,  
* `last_update` DATETIME - datum poslední změny hodnot záznamu v UTC  

Do tabulky `products_history` zkopírujte záznam z tabulky `products` ve chvíli, kdy se změnil nějaký sledovaný údaj (název, cena nebo stav skladu) a je potřeba aktualizovat data v tabulce `products`. Pozor, jedno `id` může mít více variant `url` s různou cenou. Při opětovném volání funkce **sync()** se prověří existence záznamu v `products`, prověří se shoda hodnot a vždy aktualizuje hodnota `last_update`, aby bylo zřejmé, ke kterému datu je informace platná.

**Předpokládaná náročnost**: 1 hodina

### Závislosti, načtení knihoven

V následující buňce deklarujte všechny závislosti

In [1]:
#%pip install requests requests_cache bs4

import requests, requests_cache, sqlite3, random, datetime
from bs4 import BeautifulSoup
from decimal import Decimal

#pro vývoj je vhodné zapnout cache (viz přednáška), pro finalní otestovaní tento řádek zakomentujte
requests_cache.install_cache('devel') 

#nadeklarujeme si novy typ sloupce DECIMAL do sqlite3, abychom měli automatický převod mezi SQLite3 a Python
from decimal import Decimal
sqlite3.register_adapter(Decimal, lambda d: str(d))
sqlite3.register_converter("DECIMAL", lambda s: Decimal(s.decode('ascii')))

### Deklarace funkce

V následujícím boxu definujte funkci **sync(name)** s jedním parametrem (název souboru s DB), která provede zadanou operaci. 
Pro přístup k DB lze s ohledem na složitost zadání použít přímo funkcionalitu vestavěného modulu sqlite3 (viz https://docs.python.org/2/library/sqlite3.html).

**TIP**: pro získání seznamu všech produktů lze použít endpoint https://www.alza.cz/Services/EShopService.svc/Filter

Mohlo by se také hodit: https://curl.trillworks.com/

In [6]:
# V tomto boxu pouze implementujte funkci ale nevolejte ji (pro vývoj si vytvořte vlastní buňky).
# nezapomeňte na cookies a hlavičky, jinak se Vám může zobrazit otázka "nejste robot?"
def sync(dbfile='data.sqlite'):
    with sqlite3.connect(dbfile, detect_types=sqlite3.PARSE_DECLTYPES) as conn:
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS products
                  (id TEXT, url TEXT, title TEXT, price DECIMAL, stock_state TEXT, last_update DATETIME, PRIMARY KEY(id,url))''')
        
        c.execute('''CREATE TABLE IF NOT EXISTS products_history
                  (id TEXT, url TEXT, url_alt TEXT, title TEXT, price DECIMAL, stock_state TEXT, last_update DATETIME)''')

        c.execute('''CREATE INDEX IF NOT EXISTS idx_id ON products (id)''')
        c.execute('''CREATE INDEX IF NOT EXISTS idx_idurl ON products_history (id, url)''')

        s = requests.session()
        
        #zde dopiste kod, predpokladana delka cca 50 radku
        headers = {
    'authority': 'www.alza.cz',
    'cache-control': 'max-age=0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'referer': 'http://localhost:8888/',
    'accept-language': 'sk-SK,sk;q=0.9,cs;q=0.8,en-US;q=0.7,en;q=0.6',
    'cookie': '__uzma=1f22a0b0-baa6-4fa9-a71d-451e2c8fcd6a; __uzmb=1602620116; __uzme=4861; VZTX=2733254163; TPL=1; CCC=18863907; CriticalCSS=6858194; .AspNetCore.Culture=c%3Dcs-CZ%7Cuic%3Dcs-CZ; _vwo_uuid_v2=D1424839DBF8E18D19F6126EFE4B45DC9|c4261905652b4bb599d831078fc529a4; __ssds=2; __ssuzjsr2=a9be0cd8e; __uzmaj2=872c22d0-babe-4a28-a326-b816a5c808b0; __uzmbj2=1602677383; i18next=cs-CZ; _gid=GA1.2.483098774.1602677384; ai_user=02ajXPMWLYcA82LQmcgmKt|2020-10-14T12:09:44.442Z; _gcl_au=1.1.613976267.1602677389; db_ui=c811ca53-36f3-5c15-45e5-355854a4ebe2; _hjTLDTest=1; _hjid=a08844ad-48f2-4648-b256-ec75750846ce; _hjIncludedInSessionSample=1; _hjAbsoluteSessionInProgress=1; db_uicd=454c9966-c325-ef24-03c8-dd90e750ee8a; PVCFLP=5; __uzmcj2=664922256883; __uzmdj2=1602677571; lb_id=b0ae2fcde80aefdf082845c369118483; SL_C_23361dd035530_KEY=178242482b9ce6820d21aac111dd0e23835adf09; __uzmc=956408239211; __uzmd=1602678241; _ga=GA1.1.1771958577.1602677384; _ga_FGLGFS7LP0=GS1.1.1602677388.1.1.1602678264.60; SL_C_23361dd035530_SID=5C40vMURNg; SL_C_23361dd035530_VID=-ibqW9xeUh6; ai_session=XhPbnATpFs7KMd15JD5gZl|1602677386866|1602678448923',
}
        

        data = BeautifulSoup(requests.get('https://www.alza.cz/bezzrcadlovky-bez-objektivu/18863907.htm', headers=headers).text)
        
        #spracuj stranku
        while(True):
            htm = data.find_all('a', class_='pc browsinglink')
            for a in htm:
                # id url title price stock
                price = a['data-impression-metric2'].split() #price
                price = Decimal(price[0].replace(',','.'))
                item = (a['data-impression-id'],a['href'],a['data-impression-name'],price, a['data-impression-dimension13'])               
                
                last_update = datetime.datetime.now()
               
                c.execute("SELECT * FROM products WHERE id=? AND url=?",(item[0],item[1],))
                r = c.fetchone()

                #pridaj do tabuliek ak tam este nie su
                if (not r):
                    c.execute("INSERT INTO products (id,url,title,price,stock_state,last_update) \
      VALUES (?, ?, ?, ?, ?,?)", (item[0], item[1], item[2], price, item[3], item[4])) 
                    c.execute("INSERT INTO products_history (id,url,title,price,stock_state,last_update) \
      VALUES (?, ?, ?, ?, ?,?)", (item[0], item[1], item[2], price, item[3], item[4]))
                # aktualizuj tabulky ak sa nieco zmeni
                else:
                    c.execute("UPDATE products SET id=?,url=?,title=?,price=?,stock_state=?,last_update=? WHERE id=? AND url=?", (item[0], item[1], item[2], price, item[3], item[4],r[0],r[1]))
                    c.execute("UPDATE products_history SET id=?,url=?,title=?,price=?,stock_state=?,last_update=?",(r[0],r[1],r[2],r[3],r[4],last_update,))


            # ziskanie dalsej stranky
            next = data.find('a', class_='js-button-more button-more btnx normal')
            if (not next):
                break
            data = BeautifulSoup(requests.get('https://www.alza.cz/bezzrcadlovky-bez-objektivu/'+next['href'], headers=headers).text)
        

        conn.commit()
        c.close()

In [7]:
sync('data.sqlite')

### Ověření korektní funkce

Na následujícím kódu lze ověřit základní funkcionalitu. Měly byste dostat stejný výstup jako je v ukázce. Protože se však stav e-shopu může měnit, uzpůsobte si eventuelně dotaz dle potřeb. Momentálně se testuje existence produktu https://www.alza.cz/sony-alpha-7ii?dq=2286288 ev. 
https://www.alza.cz/kod/OS072i1p5.

Při ověřování korektní funkce Vaší implementace bude porovnán obsah DB vytvořený Vaší funkcí s předpokládaným obsahem DB v určitou dobu a poté znovu s několika hodinovým odstupem.

In [8]:
from contextlib import closing

sync('data.sqlite')

with sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_DECLTYPES) as conn:
    with closing(conn.cursor()) as c:
        c.execute('SELECT id, url, price FROM products WHERE id=? AND url=? AND price>20000', ('OS072i1p5','/sony-alpha-7ii?dq=2286288'))
        r = c.fetchone()
        print(r)
        assert(r != None)

        c.execute('SELECT id, url, price FROM products WHERE id=? AND price>30000', ('OF7032a',))
        r = c.fetchall()
        print(r)
        assert (len(r)>0 and '/fujifilm-x-t3?dq=5457426' in [a[1] for a in r])

print("OK")        

('OS072i1p5', '/sony-alpha-7ii?dq=2286288', Decimal('31490'))
[('OF7032a', '/fujifilm-x-t3?dq=5457426', Decimal('39990')), ('OF7032a', '/fujifilm-x-t3-telo-cerny-levne-d5754350.htm', Decimal('36989.7')), ('OF7032a', '/fujifilm-x-t3-telo-cerny-sleva-d5877920.htm', Decimal('33990.11'))]
OK


### Komentář
Do pole níže můžete vložit textový komentář týkající se tohoto úkolu. Např. jak dlouho Vám trvalo řešení, co bylo obtížné, co bylo se mělo více v rámci přenášky vysvětlit apod.

n/a